# Ch12. 축구 A매치 데이터로 다양한 집계하기

1993 ~ 2022 축구 A매치 데이터를 사용한다.

## 1. 전 세계 데이터를 value_counts로 간단히 집계하기

In [ ]:
# 프로젝트 코드
import pandas as pd
pd.options.display.max_rows = 6
pd.options.display.float_format = '{:.2f}'.format
url = './12_01_soccer_match.csv'

In [ ]:
# date 열을 datetime으로 적용해 데이터 프레임부르기
df = pd.read_csv(url, parse_dates=['date'])
df

df의 열
- date : 경기 날짜
- nation : 경기를 치른 국가
- continent : nation 열의 국가의 소속대륙
- H/A : nation 열의 국가가 Home인지 Away인지 여부
- oppenent : 상대팀 
- o_continent : 상대팀의 소속대륙
- score : nation 열의 국가의 득점
- o_score : 상대팀의 득점
- tournament : 경기 구분(Friendly : 친선, WC : 월드컵, Others: 그외)
- result : 경기 결과

In [ ]:
df.info()

국가별 A매치 경기수와 총 득점 집계하기

In [ ]:
# 경기수가 많은 나라 top5
df['nation'].value_counts()[:5]

In [ ]:
# 경기수가 많은 나라 top5 그래프
df['nation'].value_counts()[:5].plot(kind='bar')

In [ ]:
# 승리 많은 나라 top5
cond1 = df['result'] == 'Win'
df.loc[cond1, 'nation'].value_counts()[:5]

In [ ]:
# 승리 많은 나라 top5 그래프
df.loc[cond1, 'nation'].value_counts()[:5].plot(kind='bar')

## 2. 전 세계 데이터를 groupby로 다양한 집계하기

In [ ]:
# 프로젝트 코드
import pandas as pd
pd.options.display.max_rows = 6
pd.options.display.float_format = '{:.2f}'.format
url = './12_01_soccer_match.csv'

# date 열을 datetime으로 적용해 데이터 프레임부르기
df = pd.read_csv(url, parse_dates=['date'])
df

국가별 누적 득점 집계하기

In [ ]:
# 누적 득점 많은 나라 top5
df.groupby('nation')['score'].sum().sort_values(ascending=False)[:5]

In [ ]:
# 누적 득점 많은 나라 top5 그래프
df.groupby('nation')['score'].sum().sort_values(ascending=False)[:5].plot(kind='bar')

이제껏 배운 것을 복습할겸 정렬을 하고 슬라이싱을 했지만 그것이 번거롭다면 간편한 함수 `nlargest`를 쓸 수 있다

작은 순서대로 뽑을 때는 `nsmallest`를 쓸 수 있다

- pandas 공식문서 nlargest : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nlargest.html
- pandas 공식문서 nsmallest : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nsmallest.html

In [ ]:
# 간편한 함수 nlargest를 쓸 수도 있다
df.groupby('nation')['score'].sum().nlargest(5)

국가별 평균 득점 집계하기

In [ ]:
# 평균 득점이 높은 나라 top 5
df.groupby('nation')['score'].mean().sort_values(ascending=False)[:5]

New Caledonia 와 같이 경기수가 적은 나라가 평균 득점 상위에 오르는 것을 방지하기 위해 

A매치 100경기 이상의 국가들만 평균 득점을 집계하겠다

In [ ]:
# 총 게임수가 100경기 이상의 국가들만의 데이터
cond2 = df.groupby('nation')['score'].transform('count') >= 100
df[cond2]

In [ ]:
# 100경기 이상 경기한 나라 중 평균 득점이 높은 5개국
df[cond2].groupby('nation')['score'].mean().sort_values(ascending=False)[:5]

In [ ]:
# 100경기 이상 경기한 나라 중 평균 득점이 높은 나라 그래프
df[cond2].groupby('nation')['score'].mean().sort_values(ascending=False)[:5].plot(kind='bar')

월드컵 다득점 국가 집계하기

In [ ]:
# 전체 데이터에서 월드컵 관련 데이터만 필터링
cond3 = df['tournament'] == 'WC'
df[cond3]

In [ ]:
# 월드컵 다득점 상위 5개국
df[cond3].groupby('nation')['score'].sum().sort_values(ascending=False)[:5]

In [ ]:
# 월드컵 다득점 상위 5개국 그래프
df[cond3].groupby('nation')['score'].sum().sort_values(ascending=False)[:5].plot(kind='bar')

## 3. 한국축구 데이터를 groupby와 resample로 집계하기

In [ ]:
# 프로젝트 코드
import pandas as pd
pd.options.display.max_rows = 6
pd.options.display.float_format = '{:.2f}'.format
url = './12_01_soccer_match.csv'

# date 열을 datetime으로 적용해 데이터 프레임부르기
df = pd.read_csv(url, parse_dates=['date'])
df

In [ ]:
# 한국의 데이터만 필터링하고 DatetimeIndex로 만들기
cond4 = df['nation'] == 'Korea'
df_kr = df[cond4].set_index('date')
df_kr

In [ ]:
# 한국 데이터 중 2002년도의 데이터만 필터링
df_kr.loc['2002']

In [ ]:
# 승률을 구하기 위한 lambda 함수 만들기 위해 시리즈 생성
x = pd.Series(['Win', 'Lose', 'Win', 'Draw'])
x

In [ ]:
# x의 승률은 다음과 같다
(x == 'Win').mean()

In [ ]:
# 비교연산 == 대신 연산함수 eq를 쓰면 좀 더 코드가 깔끔하다 (실습은 ==로 진행)
x.eq('Win').mean()

In [ ]:
# 상대 대륙별 한국팀의 승률
df_kr.groupby('o_continent')['result'].agg(lambda x: (x == 'Win').mean())

In [ ]:
# 승률을 구하는 lambda 함수는 자주 쓰게 될것 같으니 함수 wp로 선언하자
wp = lambda x: (x == 'Win').mean()
df_kr.groupby('o_continent')['result'].agg(wp)

In [ ]:
# 한국의 상대 대륙별 승률과 평균 득점을 동시에
wp = lambda x: (x == 'Win').mean()
(df_kr
 .groupby('o_continent').agg({'result': wp, 'score': 'mean'})
 .set_axis(['wp', 'goal per game'], axis=1))

In [ ]:
# 연도별 한국의 승률
df_kr.resample('Y')['result'].agg(wp)

In [ ]:
# 연도별 한국의 승률 그래프
df_kr.resample('Y')['result'].agg(wp).plot()

In [ ]:
# 2002년의 승률 확인
df_kr.loc['2002', 'result'].value_counts(normalize=True)

In [ ]:
# 연도별 한국의 평균 득점
df_kr.resample('Y')['score'].mean()

In [ ]:
# 연도별 한국의 평균 득점 그래프
df_kr.resample('Y')['score'].mean().plot()